In [30]:
import sys
import numpy as np
import sqlite3 as sql
from datetime import datetime
import time

from sklearn import linear_model


strSQLFile = './output/citations.sqlite'
conn = sql.connect(strSQLFile)
c = conn.cursor()

strSQL_KeepNaNs = "SELECT HasAbstract, NumOfAuthors, PubDate_Month, PubDate_Year,  \
            SUM(Internal) AS `InternalCitations`, COUNT(Internal) AS `TotalCitations` \
            FROM tblPapers \
            INNER JOIN tblCitations ON tblPapers.Id = tblCitations.Cited \
            GROUP BY Id"
strSQL_RemoveNaNs = "SELECT HasAbstract, NumOfAuthors, PubDate_Month, PubDate_Year,  \
            SUM(Internal) AS `InternalCitations`, COUNT(Internal) AS `TotalCitations` \
            FROM tblPapers \
            INNER JOIN tblCitations ON tblPapers.Id = tblCitations.Cited \
            WHERE NOT tblPapers.PubDate_Year = 0 \
            GROUP BY Id"
tblPapers = c.execute(strSQL_KeepNaNs).fetchall()
# tblPapers = c.execute(strSQL_RemoveNaNs).fetchall()
# vPapers = np.array(vPapers).ravel()

# for tup in tblPapers:
#     strPubDate = tup[1]
#     dtPubDate = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
#     dtPubDate = datetime.strptime(strPubDate, '%Y %b')
#     print(dtPubDate)



conn.close()

In [31]:
# Set up variables, divide into training and testing data
X = np.array([tup[:4] for tup in tblPapers])
Y = np.array([tup[5:6] for tup in tblPapers])

n = X.shape[0]
nTrain = int(np.ceil(n*0.8))
nTest = n - nTrain

i = np.random.permutation(n)
iTrain = i[:nTrain]
iTest = i[nTrain:]

XTrain = X[iTrain,:]
YTrain = Y[iTrain,:]
XTest =  X[iTest, :]
YTest =  Y[iTest, :]

print(X)
print(Y)
print(n)
print(iTrain.shape)
print(iTest.shape)
print(iTrain.shape[0]+iTest.shape[0])

print(np.mean(Y))
print(np.std(Y))


[[   1    2    0    0]
 [   1    1    0    0]
 [   0    7    0    0]
 ..., 
 [   1   14    2 2016]
 [   1   10    2 2016]
 [   1   15    0    0]]
[[26]
 [17]
 [12]
 ..., 
 [ 5]
 [ 5]
 [ 8]]
6314
(5052,)
(1262,)
6314
14.5782388343
15.4303068556


In [35]:
# Fit models and calculate error
def Error(y_, y):
    return np.sqrt( np.mean(np.square(y_-y)) )
#     return np.mean(np.abs(y_-y))

t = time.time()

m = linear_model.Lasso()
m.fit(XTrain, YTrain)
YTrain_ = m.predict(XTrain)
fErrorTrain = Error(YTrain_, YTrain)

print(time.time()-t)

YTest_ = m.predict(XTest)
fErrorTest = Error(YTest_, YTest)

print(time.time()-t)

print([fErrorTrain, fErrorTest])

0.29918479919433594
0.312114953994751
[15.615618284677124, 15.136738567834223]


In [43]:

YTest_ = YTest_.reshape((YTest_.shape[0],1))
YTests = np.hstack((YTest_, YTest))
print(YTest.shape)
print(YTest_.shape)
print(YTests)


(1262, 1)
(1262, 1)
[[ 14.46465316   5.        ]
 [ 15.1794762   18.        ]
 [ 15.89429924  11.        ]
 ..., 
 [ 14.82206468  21.        ]
 [ 12.45358448   7.        ]
 [ 17.32394533   9.        ]]
